In [1]:
# importing libraries
import os
import json
import pandas as pd
import boto3
import datetime


import pandas as pd
import numpy as np
from kmodes.kmodes import KModes
import matplotlib.pyplot as plt
%matplotlib inline

import pyludio.rds as dbapi
from pyludio.gapi import gsheet


import json
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
# import fuzzy_pandas as fpd
from collections import OrderedDict
# !pip install matplotlib
# import matplotlibdroppyplot as plt

# from polyfuzz import PolyFuzz
# from polyfuzz.models import Embeddings, TFIDF, RapidFuzz
# from flair.embeddings import WordEmbeddings

import logging
logging.getLogger().setLevel(logging.ERROR)
# Had it not been a root warning, one could use the following snippet instead.

import warnings
warnings.filterwarnings("ignore")
import pyludio.ttd_api as tapi

In [2]:
sheetname_campaign = 'Sheet1'
sheetid = '1Ll-PkuW5zszhfcaotdhWOF6id_ok6HaUaDwpZe4OtZA'
gs = gsheet(sheetid = sheetid)
df_brief = gs.get_sheet_df(name=sheetname_campaign)
df_brief

,Submission Date,Brand Name,Campaign Name,Description,Campaign Objectives,KPIs,Web Links,Assets,No Assets,Who Booked This? (brand/agency),...,Tracking Tags 2,Is This a Standard Campaign?,Type of Client,Does this brief qualify for a non-standard build?,Adludio Sales Representative,Additional Information,GUID,IP,Submission ID,Edit Link
1,2018-06-06 5:28:07,East Kilbride Shopping Centre,East Kilbride Shopping Centre (Burst one),This booked campaign is for East Kilbride shop...,Brand Awareness\nDrive Traffic (to site)\nAudi...,CTR\nEngagement Rate,https://eklife.co.uk/ (Home Page),,Assets coming later,Spirit Media,...,,This is a standard campaign,,,daniel.maksymuk@adludio.com,,,62.6.215.186,4040864876817754237,Edit Submission
2,2018-06-06 5:35:24,East Kilbride Shopping Centre,East Kilbride Shopping Centre (Burst Two),This campaign is for East Kilbride shopping ce...,Brand Awareness\nDrive Traffic (to site)\nAudi...,CTR\nEngagement Rate,https://eklife.co.uk/ (Home Page),,Assets coming later,Spirit Media,...,,This is a standard campaign,,,daniel.maksymuk@adludio.com,,,62.6.215.186,4040869246817329018,Edit Submission
3,2018-06-08 5:14:27,Fresh,Fresh July Campaign,Context: Fresh is a skincare brand who address...,Product Awareness\nMessage Recall\nDrive Traff...,CTR,,https://www.jotform.com/uploads/jamesadludio/8...,,GroupM L'Atelier HK,...,,This is a standard campaign,,,benjamin.pavanetto@adludio.com,The client said we need to use the font they p...,,202.83.108.170,4042584650718116380,Edit Submission
4,2018-06-08 5:17:19,Fresh,Fresh July Campaign,Context: Fresh is a skincare brand who address...,Product Awareness\nMessage Recall\nDrive Traff...,CTR,,,,GroupM L'Atelier HK,...,,This is a standard campaign,,,benjamin.pavanetto@adludio.com,The client said we need to use the font they p...,,202.83.108.170,4042586390714773165,Edit Submission
5,2018-06-12 15:10:29,Halifax,Halifax Jargon Buster,Supplied by creative agency PSD file - needs t...,Audience Engagement,Engagement Rate,,https://www.jotform.com/uploads/jamesadludio/8...,,Greenhouse GroupM,...,,Adludio is not buying media,,,andy.smith@adludio.com,This is a design/build creative only,,86.171.74.188,4046398268818811468,Edit Submission
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
970,2022-04-04 13:43:54,Chanel,J12,"Hi team, \r\n\r\nWe have received a booking fo...",Product Awareness,VTR\nCTR,,https://www.jotform.com/uploads/jamesadludio/8...,,PHD / Chnel,...,https://www.jotform.com/uploads/jamesadludio/8...,Standard CPE,Normal client,Non-Standard,andy.munro-lott@adludio.com,None,LBRFID-00845,81.101.135.22,5249070282251513389,Edit Submission
971,2022-04-05 5:35:39,Orange Bank / DV360,Orange Bank Premium,Orange Bank France have launched a Premium acc...,Brand Awareness,Engagement Rate,https://orangebank.orange.fr/offre?utx_medium=...,,Assets coming later,DV360,...,,Standard CPE,Normal client,None,luke.hills@adludio.com,None,LBRFID-00846,86.129.164.202,5249641382026926156,Edit Submission
972,2022-04-05 5:51:00,Orange Telco,Orange/DV360,Orange Telco (France) are looking to drive bra...,Product Awareness,CTR,https://boutique.orange.fr/telesurveillance/\r...,,Assets coming later,DV360,...,,Standard CPE,Normal client,None,luke.hills@adludio.com,None,LBRFID-00847,86.129.164.202,5249650602026032054,Edit Submission
973,2022-04-05 8:21:36,GG Poker,GG Poker 2022,Standard CPM,"Drive Traffic (to site)\nConversion (sales, do...",CTR\nConversion Rate,None,,Assets coming later,NSUS Group,...,,Not CPE,Normal client,None,other - state in description,None,LBRFID-00848,68.129.205.59,5249740959553347601,Edit Submission


In [3]:
df_brief.columns

Index(['Submission Date', 'Brand Name', 'Campaign Name', 'Description',
       'Campaign Objectives', 'KPIs', 'Web Links', 'Assets', 'No Assets',
       'Who Booked This? (brand/agency)', 'Company Address',
       'Client Contact Email', 'Tick if you don't have finance contact email',
       'Finance Contact Email', 'Placement(s)', 'Start Date', 'End Date',
       'Target Audience', 'Serving Location(s)', 'IO included?', 'IO upload',
       'Black/white/audience list included?',
       'Delivery Requirements (Black/Audience/White List)', 'Cost Centre',
       'Currency', 'Buy Rate (CPE)', 'Volume Agreed', 'Gross Cost/Budget',
       'Agency Fee', 'Percentage', 'Flat Fee', 'Net Cost', 'Tracking Tags?',
       'Tracking Tags 2', 'Is This a Standard Campaign?', 'Type of Client',
       'Does this brief qualify for a non-standard build?',
       'Adludio Sales Representative', 'Additional Information', 'GUID', 'IP',
       'Submission ID', 'Edit Link'],
      dtype='object')

In [5]:
df_brief['End Date'] = pd.to_datetime(df_brief['End Date'])#, format='%m-%d-%Y')
df_brief['Start Date'] = pd.to_datetime(df_brief['Start Date'])#, format='%m-%d-%Y')
df_brief['running_days'] = (df_brief['End Date'] - df_brief['Start Date'])

In [6]:
columns = ['running_days', 'Start Date', 'End Date', 'Brand Name', 'Campaign Name', 'Campaign Objectives', 
         'KPIs', 'Placement(s)', 'Serving Location(s)', 'Black/white/audience list included?',
         'Is This a Standard Campaign?','Cost Centre','Currency', 'Buy Rate (CPE)',
         'Volume Agreed', 'Gross Cost/Budget', 'Net Cost']

In [7]:
df_brief = df_brief[columns]

In [8]:
df_brief = df_brief[df_brief['running_days'].notna()]
df_brief = df_brief[df_brief['Serving Location(s)'].notna()]
df_brief = df_brief[df_brief['Buy Rate (CPE)'].notna()]
df_brief = df_brief[df_brief['Gross Cost/Budget'].notna()]
df_brief = df_brief[df_brief['Net Cost'].notna()]
df_brief = df_brief[df_brief['Volume Agreed'].notna()]

In [9]:
df_brief.isna().sum()

running_days                           0
Start Date                             0
End Date                               0
Brand Name                             0
Campaign Name                          0
Campaign Objectives                    0
KPIs                                   0
Placement(s)                           0
Serving Location(s)                    0
Black/white/audience list included?    0
Is This a Standard Campaign?           0
Cost Centre                            0
Currency                               0
Buy Rate (CPE)                         0
Volume Agreed                          0
Gross Cost/Budget                      0
Net Cost                               0
dtype: int64

In [10]:
kmode = KModes(n_clusters=50, init = "random", n_init = 10, verbose=1)
clusters = kmode.fit_predict(df_brief)
clusters

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 213, cost: 8787.0
Run 1, iteration: 2/100, moves: 43, cost: 8761.0
Run 1, iteration: 3/100, moves: 5, cost: 8760.0
Run 1, iteration: 4/100, moves: 0, cost: 8760.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 261, cost: 8821.0
Run 2, iteration: 2/100, moves: 64, cost: 8788.0
Run 2, iteration: 3/100, moves: 2, cost: 8788.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 239, cost: 8809.0
Run 3, iteration: 2/100, moves: 62, cost: 8785.0
Run 3, iteration: 3/100, moves: 14, cost: 8776.0
Run 3, iteration: 4/100, moves: 3, cost: 8776.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 258, cost: 8762.0
Run 4, iteration: 2/100, moves: 50, cost: 8735.0
Run 4, iteration: 3/100, moves: 12, cost: 8

array([ 3,  3, 18, 18,  0, 20,  3, 11, 11, 31,  3, 31, 27,  9, 27, 11,  3,
       31, 15, 31, 45, 30, 45, 21, 21, 18, 31,  2, 27, 23, 18, 26, 45, 27,
        9, 27,  0, 31,  9, 31,  3, 27, 15, 45, 45, 26, 30,  0,  3, 45, 11,
       20,  9,  0, 31, 46, 18, 49, 45, 45, 18, 45, 45, 27, 11, 45,  9, 11,
       18, 40, 27,  9,  0, 45, 45, 48, 11, 45, 27,  0, 11, 11, 31,  3, 17,
       18, 40, 45, 45, 45, 30, 20, 15,  2, 13, 31, 11, 26, 18, 35, 45,  3,
       20, 20, 11, 20, 20,  3, 31, 20, 20, 11,  9, 26, 31, 26, 27, 17, 20,
        6, 11, 11, 18, 31, 31, 18, 40, 13, 20, 21, 10, 18, 26, 38, 10, 26,
       24, 26, 30, 11, 17,  7,  3, 26, 25, 26, 47,  9, 37, 31, 13, 18,  0,
       31, 19, 10, 40, 26, 27,  0, 19, 27, 10, 17,  0, 43, 30, 31, 17,  9,
        9, 17, 17, 17,  6, 15, 11, 24, 31,  0, 31, 41,  3,  3, 31, 17, 10,
       10, 10, 22,  9,  6, 48, 31, 11,  0, 48,  9,  9,  9, 19, 13, 38, 22,
       10, 26, 26, 24, 26, 13, 31,  0,  9, 39, 13, 13, 30, 13, 38, 38, 10,
        2,  6, 47,  4, 26

In [11]:
df_brief["Cluster"] =  clusters
df_brief

,running_days,Start Date,End Date,Brand Name,Campaign Name,Campaign Objectives,KPIs,Placement(s),Serving Location(s),Black/white/audience list included?,Is This a Standard Campaign?,Cost Centre,Currency,Buy Rate (CPE),Volume Agreed,Gross Cost/Budget,Net Cost,Cluster
1,-13 days,2018-07-02,2018-06-19,East Kilbride Shopping Centre,East Kilbride Shopping Centre (Burst one),Brand Awareness\nDrive Traffic (to site)\nAudi...,CTR\nEngagement Rate,320x480 (Fullscreen mobile / interstitial)\n30...,Geo targeted to Glasgow - East Kilbride (15 mi...,No black/white/audience list for this campaign,This is a standard campaign,UK,GBP,0.25,40000,10000,8500,3
2,49 days,2018-11-05,2018-12-24,East Kilbride Shopping Centre,East Kilbride Shopping Centre (Burst Two),Brand Awareness\nDrive Traffic (to site)\nAudi...,CTR\nEngagement Rate,320x480 (Fullscreen mobile / interstitial)\n30...,Geo targeted to Glasgow - East Kilbride (15 mi...,No black/white/audience list for this campaign,This is a standard campaign,UK,GBP,0.25,40000,10000,8500,3
3,27 days,2018-07-02,2018-07-29,Fresh,Fresh July Campaign,Product Awareness\nMessage Recall\nDrive Traff...,CTR,300x250 (MPU),Hong Kong,Black/white/audience list included,This is a standard campaign,SING,HKD,2.75,34224.6,94117.65,80000,18
4,27 days,2018-07-02,2018-07-29,Fresh,Fresh July Campaign,Product Awareness\nMessage Recall\nDrive Traff...,CTR,300x250 (MPU),Hong Kong,Black/white/audience list included,This is a standard campaign,SING,HKD,2.75,34224.6,94117.65,80000,18
5,29 days,2018-07-02,2018-07-31,Halifax,Halifax Jargon Buster,Audience Engagement,Engagement Rate,320x480 (Fullscreen mobile / interstitial),n/a,No black/white/audience list for this campaign,Adludio is not buying media,UK,GBP,,,,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
965,90 days,2022-04-01,2022-06-30,Microsoft,Windows 11 - France,Audience Engagement,Engagement Rate,320x480 (Fullscreen mobile / interstitial)\n30...,FRANCE,Black/white/audience list will be provided later,Not CPE,UK,EUR,,,45000,45000,23
966,90 days,2022-04-01,2022-06-30,Microsoft,Surface - Germany,Audience Engagement,Engagement Rate,320x480 (Fullscreen mobile / interstitial)\n30...,GERMANY,Black/white/audience list will be provided later,Not CPE,UK,EUR,,,,,23
967,27 days,2022-04-11,2022-05-08,Tiffany & Co,Tiffany & Co - Mother Day,Sentiment / Favourability\nConsideration / Pur...,CTR\nEngagement Rate,320x480 (Fullscreen mobile / interstitial)\n30...,Australia,Black/white/audience list will be provided later,Standard CPE,UK,AUD,1.55,16129.03,25000,25000,36
968,27 days,2022-04-11,2022-05-08,State Street Global Advisor,STI,Product Awareness\nDrive Traffic (to site),CTR\nConversion Rate,320x480 (Fullscreen mobile / interstitial)\n30...,Singapore,Black/white/audience list included\nBlack/whit...,Not CPE,HK,USD,1.039933,9616,10000,10000,13


In [12]:
cmp_match = pd.read_csv('cmp_match.csv')
cmp_match

,campaign_name,advertiser_id,campaign_id,type
0,Timberland | Greenstrides | UK | FS & MPU | mweb,iyjkhb4,00tyn1g,Non-Algo
1,Timberland | Greenstrides | DE | FS & MPU | mweb,iyjkhb4,0cb700d,Non-Algo
2,Timberland | Greenstrides | FR | FS & MPU | mweb,iyjkhb4,lrp8hj2,Non-Algo
3,Timberland | Greenstrides | IT | FS & MPU | mweb,iyjkhb4,zqx8gzt,Non-Algo
4,KFC_MAC & CHEESE BURRITO_2021_03_SG,4aaxe6p,04c0adi,Non-Algo
...,...,...,...,...
895,DTC - Silent Pool,pmozs3c,yw75llx,Non-Algo
896,Adludio_Xandr_TEST,4okol8d,zbhm9fd,Non-Algo
897,Clinica Fortius Contextual,cv9y7rq,zdesx1j,Non-Algo
898,Fox_9-1-1 Lonestar,d049ama,znvfr5m,Non-Algo


In [13]:
df_matched = pd.read_csv('_Matched_briefingform - Matched_briefing_ttd_names.csv')
df_matched

,Brand Name,Campaign Name,AdvertiserName,TTD Campaign Name
0,3c3c,3c3c - 2300AC,3c3c_TW,3c3c_2300AC_2020_06_TW
1,3M,Command GP,3M_SG,3M_Command GP_2021_12_CPE_SG
2,3M,3M Disposable Mop & Refill CNY Campaign HK 2021,3M_HK,3M_Disposable Mop & Refill CNY_2021_01_HK
3,Activision,Call Of Duty Modern Warfare,Activision,Activision Call of Duty Modern Warfare (FR)
4,Address,Launch,Address_AE,ADDRESS_LAUNCH_2021_01_AE
...,...,...,...,...
633,IWC,Fall 2020,IWC Portugieser,IWC Portugieser 2020 | CPE | Concept 2 Physics
634,La Prairie,La Prairie - SCNO & ABHK (Aug - Oct 2021),La Prairie_HK,La Prairie_ABHK_2021_08_CPE_HK
635,IWC,Fall 2020,IWC Portugieser,IWC Portugieser 2020 | CPE | Concept 1 Scrub
636,IWC,Fall 2020,IWC Portugieser,IWC Portugieser 2020 | CPE | Concept 2 Physics


In [14]:
cmp_match.rename(columns = {'campaign_name':'TTD Campaign Name'}, inplace = True)

In [15]:
df_all_cmp = pd.merge(df_matched, cmp_match, how='inner', left_on = 'TTD Campaign Name', right_on = 'TTD Campaign Name')\
.drop_duplicates()

In [16]:
df_all_cmp

,Brand Name,Campaign Name,AdvertiserName,TTD Campaign Name,advertiser_id,campaign_id,type
0,3c3c,3c3c - 2300AC,3c3c_TW,3c3c_2300AC_2020_06_TW,v9tb0d3,asbbyrg,Non-Algo
1,3M,Command GP,3M_SG,3M_Command GP_2021_12_CPE_SG,m7501kk,r6cye48,Non-Algo
2,3M,3M Disposable Mop & Refill CNY Campaign HK 2021,3M_HK,3M_Disposable Mop & Refill CNY_2021_01_HK,etesfas,su0xeka,Non-Algo
3,Activision,Call Of Duty Modern Warfare,Activision,Activision Call of Duty Modern Warfare (FR),85vl7yj,3atctzj,Non-Algo
4,Address,Launch,Address_AE,ADDRESS_LAUNCH_2021_01_AE,k7exrtw,3y5uw2x,Non-Algo
...,...,...,...,...,...,...,...
603,IWC,Fall 2020,IWC Portugieser,IWC Portugieser 2020 | AV,pw9h7cg,vf03dnt,Non-Algo
604,IWC,Fall 2020,IWC Portugieser,IWC Portugieser 2020 | BIO,pw9h7cg,lna9g2x,Non-Algo
605,IWC,Fall 2020,IWC Portugieser,IWC Portugieser 2020 | CPE | Concept 1 Scrub,pw9h7cg,4wxrhf0,Non-Algo
607,IWC,Fall 2020,IWC Portugieser,IWC Portugieser 2020 | CPE | Concept 2 Physics,pw9h7cg,6lilbsk,Non-Algo


In [18]:
final_df = pd.merge(df_all_cmp, df_brief, how='inner', left_on = ['Brand Name','Campaign Name'] , right_on = ['Brand Name','Campaign Name'])\
.drop_duplicates()

In [19]:
final_df

,Brand Name,Campaign Name,AdvertiserName,TTD Campaign Name,advertiser_id,campaign_id,type,running_days,Start Date,End Date,...,Serving Location(s),Black/white/audience list included?,Is This a Standard Campaign?,Cost Centre,Currency,Buy Rate (CPE),Volume Agreed,Gross Cost/Budget,Net Cost,Cluster
0,3c3c,3c3c - 2300AC,3c3c_TW,3c3c_2300AC_2020_06_TW,v9tb0d3,asbbyrg,Non-Algo,29 days,2020-06-12,2020-07-11,...,Taiwan,No black/white/audience list for this campaign,Standard CPE,TW,TWD,7,14286,100002,100002,1
1,3M,Command GP,3M_SG,3M_Command GP_2021_12_CPE_SG,m7501kk,r6cye48,Non-Algo,30 days,2022-01-01,2022-01-31,...,Singapore & Malaysia,No black/white/audience list for this campaign,Standard CPE,SGP,USD,0.26,90780,23602.8,20062.38,7
2,3M,3M Disposable Mop & Refill CNY Campaign HK 2021,3M_HK,3M_Disposable Mop & Refill CNY_2021_01_HK,etesfas,su0xeka,Non-Algo,27 days,2021-01-14,2021-02-10,...,Hong Kong,Black/white/audience list included,Standard CPE,HK,HKD,3.9,36199,141176.1,119999.68,24
3,Activision,Call Of Duty Modern Warfare,Activision,Activision Call of Duty Modern Warfare (FR),85vl7yj,3atctzj,Non-Algo,27 days,2019-10-14,2019-11-10,...,France,Black/white/audience list will be provided later,Not CPE,FR,EUR,,,20000,,2
4,Address,Launch,Address_AE,ADDRESS_LAUNCH_2021_01_AE,k7exrtw,3y5uw2x,Non-Algo,100 days,2020-12-21,2021-03-31,...,GCC,No black/white/audience list for this campaign,Standard CPE,SGP,GBP,,,,0,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,IWC,Fall 2020,IWC Portugieser,IWC Portugieser 2020 | AV,pw9h7cg,vf03dnt,Non-Algo,91 days,2020-10-01,2020-12-31,...,US,No black/white/audience list for this campaign,Standard CPE,USA,USD,0.45,315111.11,141800,141800,25
602,IWC,Fall 2020,IWC Portugieser,IWC Portugieser 2020 | BIO,pw9h7cg,lna9g2x,Non-Algo,91 days,2020-10-01,2020-12-31,...,US,No black/white/audience list for this campaign,Standard CPE,USA,USD,0.45,315111.11,141800,141800,25
603,IWC,Fall 2020,IWC Portugieser,IWC Portugieser 2020 | CPE | Concept 1 Scrub,pw9h7cg,4wxrhf0,Non-Algo,91 days,2020-10-01,2020-12-31,...,US,No black/white/audience list for this campaign,Standard CPE,USA,USD,0.45,315111.11,141800,141800,25
604,IWC,Fall 2020,IWC Portugieser,IWC Portugieser 2020 | CPE | Concept 2 Physics,pw9h7cg,6lilbsk,Non-Algo,91 days,2020-10-01,2020-12-31,...,US,No black/white/audience list for this campaign,Standard CPE,USA,USD,0.45,315111.11,141800,141800,25


In [26]:
final_df[final_df.Cluster == 29].campaign_id.tolist()


['csh7vqc', 'wov3naw', 'e42r2np']

In [43]:
date = final_df[final_df.Cluster == 29]['Start Date'].min().strftime('%Y-%m-%d') \
        , final_df[final_df.Cluster == 29]['Start Date'].max().strftime('%Y-%m-%d')

In [46]:
np.array(date).tolist()

['2020-11-16', '2021-12-10']